씐나는 과제 - 이미지 잘 찾는 모데루 만들기~  
다중분류!!

In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

import torch       
import torch.nn as nn              
import torch.nn.functional as F 
from torch.utils.data import Dataset, DataLoader 
import torch.optim as optim    
from torchmetrics.classification import F1Score, BinaryF1Score, BinaryAccuracy, Recall, MulticlassF1Score
from torchmetrics.classification import BinaryConfusionMatrix
import torch.optim.lr_scheduler as lr_scheduler

import pandas as pd 
import matplotlib.pyplot as plt              
from sklearn.preprocessing import * 
from sklearn.model_selection import train_test_split 


from sklearn.datasets import make_classification
from sklearn.decomposition import PCA


In [2]:
# 데이터셋 준비
imgDF = pd.read_csv(r'C:\Users\hoon\Desktop\KDT 6\EX_PANDAS6-main\OPENCV\data\MNIST_train.csv')


In [3]:
# 데이터셋 정보 확인
imgDF.index

RangeIndex(start=0, stop=59999, step=1)

타겟 : 1번 컬럼  
피쳐 : 나머지 전부

In [4]:
# 결측치 확인
imgDF.isna().sum()

5        0
0        0
0.1      0
0.2      0
0.3      0
        ..
0.613    0
0.614    0
0.615    0
0.616    0
0.617    0
Length: 785, dtype: int64

In [5]:
# 피쳐와 타겟 분리
featureDF = imgDF.iloc[:, 1:]  
targetDF = imgDF[[imgDF.columns[0]]]

#### 모델 구현 클래스 만들기

In [6]:
class imgmodel(nn.Module) :
    def __init__(self) :
        super().__init__()
        self.in_layer = nn.Linear(784,10)
        self.h1_layer = nn.Linear(10,30)
        self.h2_layer = nn.Linear(30,50)
        self.h3_layer = nn.Linear(50,30)
        self.h4_layer = nn.Linear(30,10)
        self.out_layer = nn.Linear(10,10)

    def forward(self,data) :
        y = F.leaky_relu(self.in_layer(data))
        
        y = F.leaky_relu(self.h1_layer(y))
        y = F.leaky_relu(self.h2_layer(y))
        y = F.leaky_relu(self.h3_layer(y))
        y = F.leaky_relu(self.h4_layer(y))

        return self.out_layer(y)

In [7]:
# 모델 인스턴스 생성
model = imgmodel()
print(model)

imgmodel(
  (in_layer): Linear(in_features=784, out_features=10, bias=True)
  (h1_layer): Linear(in_features=10, out_features=30, bias=True)
  (h2_layer): Linear(in_features=30, out_features=50, bias=True)
  (h3_layer): Linear(in_features=50, out_features=30, bias=True)
  (h4_layer): Linear(in_features=30, out_features=10, bias=True)
  (out_layer): Linear(in_features=10, out_features=10, bias=True)
)


데이터셋 클래스 만들기

In [8]:
class imgDataset(Dataset) :
    def __init__(self,featureDF,targetDF) :
        self.featureDF = featureDF
        self.targetDF = targetDF
        self.n_rows = featureDF.shape[0]
        self.n_features = featureDF.shape[1]

    def __len__(self) :
        return self.n_rows
    
    def __getitem__(self,index) :
        featureTS = torch.FloatTensor(self.featureDF.iloc[index].values)
        targetTS = torch.FloatTensor(self.targetDF.iloc[index].values)

        return featureTS, targetTS

In [9]:
# 커스텀 데이터셋 생성
imgDS = imgDataset(featureDF,targetDF)

# 데이터로더 인스턴스 생성
imgDL = DataLoader(imgDS)

학습 준비

In [10]:
EPOCH = 30
BATCH_SIZE = 3
LR = 0.001

In [11]:
# 학습용, 검증용 데이터셋 생성

X_train, X_test, y_train, y_test = train_test_split(featureDF,targetDF,
                                                    random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,
                                                  random_state=1)


print(f'{X_train.shape} {X_test.shape} {X_val.shape}')
print(f'{y_train.shape} {y_test.shape} {y_val.shape}')
print(f'{y_train.value_counts()} {y_test.value_counts()} {y_val.value_counts()}')
print(f'{type(X_train)} {type(X_test)} {type(X_val)}')

trainDS = imgDataset(X_train,y_train)
valDS = imgDataset(X_val,y_val)
testDS = imgDataset(X_test,y_test)

# 각 데이터로더 생성
trainDL = DataLoader(trainDS, batch_size=BATCH_SIZE)
valDL = DataLoader(valDS, batch_size=BATCH_SIZE)

(33749, 784) (15000, 784) (11250, 784)
(33749, 1) (15000, 1) (11250, 1)
5
1    3810
7    3531
3    3421
9    3352
0    3347
6    3342
4    3308
2    3300
8    3271
5    3067
Name: count, dtype: int64 5
1    1650
7    1564
3    1531
2    1527
8    1503
0    1474
6    1468
4    1458
9    1452
5    1373
Name: count, dtype: int64 5
1    1282
3    1179
7    1170
9    1145
2    1131
6    1108
0    1102
8    1077
4    1076
5     980
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>


In [12]:
# 최적화 준비
optimizer = optim.Adam(model.parameters(),lr=LR)

crossLoss = nn.CrossEntropyLoss()

scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode = 'max', patience=30 )


In [13]:
## 학습의 효과 확인을 위해 손실값과 성능평가값 저장 필요
LOSS_HISTORY, SCORE_HISTORY = [[],[]],[[],[]]
CNT = len(trainDL)

## 학습 모니터링/스케줄링 설정 
# => LOSS_HISTORY, SCORE_HISTROY 활용
# => 임계기준 : 10번
BREAK_CNT = 0

for epoch in range(EPOCH) :
    # 학습 모드로 모델 설정
    model.train()


    # 배치크기만큼 데이터 로딩해서 학습 진행
    loss_total , score_total = 0, 0
    for featureTS, targetTS in trainDL :
        
        # 학습 진행
        pre_y = model(featureTS)

        # 손실 계산 : nn.CrossEntropyLoss 요구사항 : 정답/타겟은 0D 또는 1D, 타입은 long
        loss = crossLoss(pre_y,targetTS.reshape(-1).long())
        loss_total += loss.item()

        # 성능 평가 계산
        score = MulticlassF1Score(num_classes=10)(pre_y,targetTS.reshape(-1))
        score_total += score.item()

        # 최적화 진행
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # 에포크 당 검증기능
    # 모델 검증 모드 설정
    model.eval()
    with torch.no_grad():
        # 검증 데이터셋
        val_featureTS = torch.FloatTensor(valDS.featureDF.values)
        val_targetTS = torch.FloatTensor(valDS.targetDF.values)
        # 추론/평가
        pre_val = model(val_featureTS)
        # 손실계산
        loss_val = crossLoss(pre_val, val_targetTS.reshape(-1).long())
        # 성능평가
        score_val = MulticlassF1Score(num_classes=10)(pre_val, val_targetTS.reshape(-1))


    # 손실값과 성능평가값 저장
    LOSS_HISTORY[0].append(loss_total/CNT) 
    SCORE_HISTORY[0].append(score_total/CNT)
    LOSS_HISTORY[1].append(loss_val) 
    SCORE_HISTORY[1].append(score_val)  

    print(f'[{epoch}/{EPOCH}]\n- [TRAIN] LOSS : {LOSS_HISTORY[0][-1]} SCORE : {SCORE_HISTORY[0][-1]}')
    print(f'- [VALID] LOSS : {LOSS_HISTORY[1][-1]} SCORE : {SCORE_HISTORY[1][-1]}')

    scheduler.step(score_val)

    if scheduler.num_bad_epochs >= scheduler.patience :
        print(f'{scheduler.patience} EPCOH 성능 개선이 없어서 조기종료')
        break   
  

[0/30]
- [TRAIN] LOSS : 0.6221491021127616 SCORE : 0.7375629662116369
- [VALID] LOSS : 0.39784306287765503 SCORE : 0.8879976272583008
[1/30]
- [TRAIN] LOSS : 0.3673663472003261 SCORE : 0.8577629653573036
- [VALID] LOSS : 0.3707354962825775 SCORE : 0.900248110294342
[2/30]
- [TRAIN] LOSS : 0.3248045515329052 SCORE : 0.8745560514940156
- [VALID] LOSS : 0.34500065445899963 SCORE : 0.9017664194107056
[3/30]
- [TRAIN] LOSS : 0.3010347296017016 SCORE : 0.882792100807031
- [VALID] LOSS : 0.3076452910900116 SCORE : 0.9162353277206421
[4/30]
- [TRAIN] LOSS : 0.28510613183922784 SCORE : 0.8875886439469125
- [VALID] LOSS : 0.32651111483573914 SCORE : 0.9138574004173279
[5/30]
- [TRAIN] LOSS : 0.2740981870759451 SCORE : 0.891116545135445
- [VALID] LOSS : 0.311529278755188 SCORE : 0.9164637923240662
[6/30]
- [TRAIN] LOSS : 0.26628924742223065 SCORE : 0.8931575327224202
- [VALID] LOSS : 0.3601195216178894 SCORE : 0.9083243608474731
[7/30]
- [TRAIN] LOSS : 0.25909027818513647 SCORE : 0.89618419937027